In [1]:
%run Imports.ipynb

/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conf = pyspark.SparkConf()

In [3]:
spark = SparkSession.builder.appName("Transformation").getOrCreate()

23/03/25 21:59:55 WARN Utils: Your hostname, Yashasvis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.232 instead (on interface en0)
23/03/25 21:59:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/25 21:59:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/25 21:59:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/25 21:59:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/25 21:59:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/03/25 21:59:57 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
df1=spark.read.parquet("/Users/yashasvi/Documents/loadDune/*")

In [5]:
df1.count()

12619626

In [7]:
df1.createOrReplaceTempView("Load")

# Removing bot activities - Activity 1

In [8]:
df_stage_1 = spark.sql("SELECT * FROM Load Where buyer NOT IN \
                       (SELECT buyer FROM (SELECT buyer, count(buyer) as counter from Load \
                       GROUP BY buyer, (EXTRACT(HOUR FROM block_time)), (EXTRACT(MINUTE FROM block_time))) temp\
                       WHERE counter > 1)")

In [9]:
df_stage_1.createOrReplaceTempView("Load1")

# Removing bot activities - Activity 2

In [10]:
df_stage_2 = spark.sql("SELECT * FROM Load1 Where buyer NOT IN \
                       (SELECT buyer FROM(SELECT a.buyer, a.original_amount, b.original_amount,(b.block_time - a.block_time) time_gap \
                        FROM Load1 a INNER JOIN Load1 b \
                        ON (a.buyer = b.seller AND a.block_time < b.block_time AND a.nft_token_id = b.nft_token_id )) \
                        WHERE (EXTRACT(MINUTES FROM time_gap)) < 1 AND (EXTRACT(HOURS FROM time_gap)) = 0 AND \
                       (EXTRACT(DAYS FROM time_gap)) = 0 ORDER BY time_gap ASC)")

In [11]:
df_stage_2.count()

23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:00:40 WARN RowBasedKeyValueBatch: Calling spill() on

3898970

In [13]:
df_stage_2.write.parquet("/Users/yashasvi/Documents/CleanDuneData/")

23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:01:36 WARN RowBasedKeyValueBatch: Calling spill() on